In [69]:
import portfolio
import pandas as pd
import sqlite3
import datetime

dtStr = '2019-11-28'

dt = datetime.datetime.strptime(dtStr, '%Y-%m-%d').date()
dtPre = dt - datetime.timedelta(days=1)
dtPreStr = datetime.datetime.strftime(dtPre, '%Y-%m-%d')

db = sqlite3.connect('AMS.db')
query = "select * from HOlding_Table where Date = '{}'".format(dtPreStr)
dfHoldPre = pd.read_sql(query, con=db)
dfMVPre = dfHoldPre[['AccountID','SymbolCode','SymbolName','CurTrade','Qty','Cost','Close','PL_Realized','PL_Interest']]
dfMVPre.rename(columns={'Qty':'QtyPre', 'Cost':'CostPre', 'Close':'ClosePre', 'PL_Realized':'PL_Realized_Pre', 'PL_Interest':'PL_Interest_Pre'}, inplace = True)
dfMVPre.insert(0,'Date',dt)
dfMVPre

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre,PL_Realized_Pre,PL_Interest_Pre


In [70]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,Qty AS QtyBuy,SettleAmt from order_table where OrderType in ('Buy','Bonus') and Date = '{}'".format(dt)
dfBuy =  pd.read_sql(query, con=db)
dfBuy = dfBuy.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
if dfBuy.empty == False:
    dfBuy.insert(0,'Date',dt)
    dfBuy['CostBuy'] = abs(dfBuy['SettleAmt'])/dfBuy['QtyBuy']
    dfBuy.drop(columns=['SettleAmt'],inplace=True)
    dfBuy.reset_index(inplace =True)
    df_MV_Buy = pd.merge(dfMVPre,dfBuy,on=['Date','AccountID','SymbolCode','SymbolName','CurTrade'],how='outer') 
else:
    df_MV_Buy = dfMVPre
    df_MV_Buy.loc[:,'QtyBuy'] = 0
    df_MV_Buy.loc[:,'CostBuy'] = 0

df_MV_Buy.fillna(0,inplace=True)
df_MV_Buy

ValueError: cannot set a frame with no defined index and a scalar

In [72]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,Qty AS QtySell,SettleAmt from order_table where OrderType='Sell' and Date = '{}'".format(dt)
dfSell =  pd.read_sql(query, con=db)
dfSell = dfSell.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
if dfSell.empty == False:
    dfSell.insert(0,'Date',dt)
    dfSell['PriceSell'] = abs(dfSell['SettleAmt'])/dfSell['QtySell']
    dfSell.drop(columns=['SettleAmt'],inplace=True)
    dfSell.reset_index(inplace =True)
    df_MV_Buy_Sell = pd.merge(df_MV_Buy,dfSell,on=['Date','AccountID','SymbolCode','SymbolName','CurTrade'],how='outer')
else:
    df_MV_Buy_Sell = df_MV_Buy
    df_MV_Buy_Sell.loc[:,'QtySell'] = 0
    df_MV_Buy_Sell.loc[:,'PriceSell'] = 0

df_MV_Buy_Sell.fillna(0,inplace=True)
df_MV_Buy_Sell

,QtyPre,CostPre,ClosePre,PL_Realized_Pre,PL_Interest_Pre,AccountID,SymbolCode,SymbolName,CurTrade,Date,QtySell,PriceSell
0,0,0,0,0,0,FUTU,8036.HK,电子交易集团,HKD,2019-11-28,-300000,-0.193577


In [32]:
query = "select AccountID,SymbolCode,SettleAmt AS MoneyOut from order_table where OrderType ='Withdraw' and Date = '{}'".format(dt)
dfMoneyOut =  pd.read_sql(query, con=db)
dfMoneyOut = dfMoneyOut.groupby(['AccountID','SymbolCode']).sum()
#dfMoneyIn.reset_index(inplace =True)
dfMoneyOut

,,MoneyOut
AccountID,SymbolCode,
FUTU,HKD,-35686


In [30]:
query = "select AccountID,SymbolCode,SettleAmt AS MoneyIN from order_table where OrderType ='Deposit' and Date = '{}'".format(dt)
dfMoneyIn =  pd.read_sql(query, con=db)
dfMoneyIn = dfMoneyIn.groupby(['AccountID','SymbolCode']).sum()
#dfMoneyIn.reset_index(inplace =True)
dfMoneyIn

""


In [33]:
query = "select AccountID,SymbolCode,SettleAmt AS Interest from order_table where OrderType ='Interest' and Date = '{}'".format(dt)
dfInterest =  pd.read_sql(query, con=db)
dfInterest = dfInterest.groupby(['AccountID','SymbolCode']).sum()
#dfMoneyIn.reset_index(inplace =True)
dfInterest

""
